Работа с изображениями с целью аугментации.

1. Изображения, на которых автомобили есть или нет. 100 изображений целевого класса и 100 нецелевого. Размер 256x256px.
2. Библиотеки: OpenCV, pillow, scikit-image, tf.image.
3. Аугментация (вращение, растяжение/сжатие, увеличение/уменьшение). Количество изображений должно увеличиться минимум в 3 раза.

In [216]:
import cv2
import gdown
import zipfile
import shutil
import os
import random

Переменные: название входной и выходной директорий

In [217]:
in_dir = 'in'
out_dir = 'out'

Класс который предоставляет рандомное число в заданном диапазоне

In [218]:
class Range:

    def __init__(self, from_value, to_value) -> None:
        self.from_value = from_value
        self.to_value = to_value

    def randomize_int(self) -> int:
        return random.randint(self.from_value, self.to_value)

    def randomize_float(self) -> float:
        return random.uniform(self.from_value, self.to_value)

Функция для поворота изображения на рандомный угол в заданном диапазоне

In [219]:
def random_rotate(image, rotate_range: Range):
    return rotate_scale(image, rotate_range.randomize_int(), 1)

Фукция для рандомного уменьшения/увеличения изображения в заданном диапазоне


In [220]:
def random_scale(image, scale_range: Range):
    return rotate_scale(image, 0, scale_range.randomize_float())

In [221]:
def rotate_scale(image, rotate_angle: int, scale_factor: float):
    (h, w) = image.shape[0:2]
    center = (w // 2, h // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, rotate_angle, scale_factor)
    return cv2.warpAffine(image, rotation_matrix, (w, h))


Функция для рандомного сжатия/растяжения изображения в заданном диапазоне

In [222]:
def random_resize(image, resize_range: Range):
    return cv2.resize(image, None, fx=resize_range.randomize_float(), fy=resize_range.randomize_float())

Функция для аугментация изображения (вращение, сжатие/растяжение, увеличение/уменьшение)

In [223]:
def augmentation(image, out_image_size, rotate_range: Range = None, scale_range: Range = None, resize_range: Range = None):
    if rotate_range is not None:
        image = random_rotate(image, rotate_range)
    if scale_range is not None:
        image = random_scale(image, scale_range)
    if resize_range is not None:
        image = random_resize(image, resize_range)
    return cv2.resize(image, out_image_size)

Функция для сохранения изображения

In [224]:
def save_image(image, filename: str):
    path = os.path.join(out_dir, filename)
    cv2.imwrite(path, image)

Функция для получения zip архива с гугл диска и распаковки файлов во входную директорию

In [225]:
def get_files_gdrive(url: str):
    temp_zip_name = 'temp.zip'
    gdown.download(url, temp_zip_name, quiet=False)
    remove_dir(in_dir)
    zipfile.ZipFile(temp_zip_name).extractall(in_dir)
    os.remove(temp_zip_name)

Дополнительные функции для работы с директориями

In [226]:
def remove_dir(path):
    if os.path.isdir(path):
        shutil.rmtree(path)

def recreate_dir(path):
    remove_dir(path)
    os.mkdir(path, mode = 0o777, dir_fd = None)

# Главная функция
Скачивает с гугл диска zip файл с картинками и распаковывает их во входную директорию
Рекурсивно проходит по всем папкам и файлам входящей директории обрабатывает файлы и сохраняет их в выходную директорию

In [227]:
def handle_files(url: str, count: int, out_image_size=(256, 256), rotate=Range(-360, 360), scale=Range(0.1, 3), resize=Range(0.1, 3)):
    """
    :param url: ссылка на zip файл на гугл диске !ссылка должна заканчиваться на download, а не на sharing!
    :param out_image_size:
    :param count: количество изменненых файлов от одного оригинального изображения
    :param rotate: диапазон вращения изображения
    :param scale: диапазон увеличения/уменьшения нижняя граница > 0
    :param resize: диапазон/сжатия расширения нижняя граница > 0
    :return: None
    """
    get_files_gdrive(url)
    recreate_dir(out_dir)
    for path_dir, d, files in os.walk(in_dir):
        for file in files:
            path = os.path.join(path_dir, file)
            img = cv2.imread(path)
            for i in range(count):
                res = augmentation(img, out_image_size=out_image_size, rotate_range=rotate, scale_range=scale, resize_range=resize)
                out_filename = f'{file.replace(".jpg", "")}_({i}).jpg'
                save_image(res, out_filename)

handle_files('https://drive.google.com/u/0/uc?id=1Sbrvsve94G6xZ2ql_qmKUptgqapxr-kp&export=download', 3)

Downloading...
From: https://drive.google.com/u/0/uc?id=1Sbrvsve94G6xZ2ql_qmKUptgqapxr-kp&export=download
To: C:\Users\Nikolay\OneDrive\Рабочий стол\DataScience\DS3onl\Horushko Nikolay\dz-9\temp.zip
100%|██████████| 24.3M/24.3M [00:02<00:00, 11.2MB/s]
